# Set Environment

## Some Preparations

Select IDE

In [ ]:
IDE = "Colab"
# IDE = "Jupyter"

Mount drive

In [ ]:
if IDE == "Colab":
  from google.colab import drive
  drive.mount("/content/drive")

Mounted at /content/drive


Change working directory

In [ ]:
import os
if IDE == "Colab":
  os.chdir("/content/drive/MyDrive/Colab Notebooks")
  !ls
else:
  !dir

 Dataset
 Ensemble.ipynb
'model_aspect_polarity_balanced_multi_branch.pt.pt 的副本.pt 的副本.pt 的副本'
 model_hw3_1_2nd_ensemble.ckpt
 NLP_2022_Spring_Sentiment_Analysis_v2.ipynb
 old
 Sentiment_Analysis
 wandb


Install packages<br/>
% affect globally，! affect locally.<br/>
https://stackoverflow.com/a/57212513/3513289<br/>

In [ ]:
if IDE == "Colab":
  %pip install --upgrade PyYAML
  %pip install wandb
  %pip install --upgrade scikit-learn
  %pip install --upgrade --pre SimpleITK --find-links https://github.com/SimpleITK/SimpleITK/releases/tag/latest
  %pip install plotly
  %pip install timm
  %pip install transformers==4.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 5.0 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.9 MB/s 
     |████████████████████████████████| 145 kB 65.0 MB/s 
     |████████████████████████████████| 181 kB 59.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=4cac3841124632ff17c9b86e4458220b7da66195fe7921d674fc9d2398eaaf7d
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab

Import packages<br/>

In [ ]:
# Import necessary packages.
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, Dataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder
# This is for the progress bar.
# https://discuss.pytorch.org/t/error-while-multiprocessing-in-dataloader/46845/9
# import tqdm.auto as tqdm
from tqdm import tqdm
import time
from datetime import datetime
import asyncio
from glob import glob
import matplotlib.pyplot as plt
import cv2
import math
from sklearn import manifold
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import pdb
import re
import wandb
import shutil
import pandas as pd
import collections
# https://stackoverflow.com/questions/47775621/interpolate-resize-3d-array
from scipy.ndimage import zoom, gaussian_filter
from scipy import signal
import importlib
import timm
from random import sample, shuffle, randrange
from transformers import AdamW, BertModel, BertForQuestionAnswering, BertForSequenceClassification, BertTokenizerFast

## Set global variables

Set global variables

In [ ]:
"""
20220520 Andy
"batch_size": 8在Tesla P100 16G會CUDA out of memory

20220529 Andy
chinese-macbert-large在resume時，會CUDA out of memory，
改用chinese-macbert-base可解決。
"""
global_config = {
    "batch_size": 4
    , "learning_rate": 5e-5
    , "epoch_num": 50
    , "early_stop": 20
    , "max_seq_len": 512
    # , "BERT_model": "hfl/chinese-macbert-large"
    , "BERT_model": "hfl/chinese-macbert-base"
    , "doc_stride": 45
}
USE_WANDB = True
DOWNLOAD = False
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
POLARITY_TXT = ["not_mentioned", "negative", "neutral", "positive"]

Check GPU

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device is {device}")
!nvidia-smi

Device is cuda
Wed Jun  8 06:27:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    12W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                         

## Set wandb

wandb resume:
1. Change RESUME to True.
2. Replace run_id with the previous one.
https://docs.wandb.ai/guides/track/advanced/resuming

In [ ]:
RESUME = True
if RESUME:
  run_id = "2ygwnfy0"
else:
  run_id = wandb.util.generate_id()
print(run_id)

2ygwnfy0


wandb initialize

In [ ]:
if IDE == "Jupyter":
  os.environ["WANDB_NOTEBOOK_NAME"] = "Sentiment_Analysis"
brief_name = "aspect_polarity_balanced_multi_branch"
if USE_WANDB:
  wandb.login()
  wandb.init(project="Sentiment_Analysis", id=run_id, resume="allow", config=global_config, entity="nlp_2022_spring_sentiment_analysis")
  if not RESUME:
    wandb.run.name = datetime.today().strftime(f"%Y%m%d-%H%M_{brief_name}")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cyx6 (nlp_2022_spring_sentiment_analysis). Use `wandb login --relogin` to force relogin


## Set random seeds

In [ ]:
def SetSeeds(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True
  np.random.seed(seed)
SetSeeds(0)

In [ ]:
class AspectSentimentAnalysis_branch(nn.Module):
  def __init__(self, max_seq_len, polarity_num):
    super().__init__()

    self.max_seq_len = max_seq_len
    self.hidden_size = 768
    self.hidden_dim = -1
    self.polarity_num = polarity_num

    # Aspect attention pooling
    self.attention_pooling = nn.Sequential()
    self.attention_pooling.add_module(name="att_fc_in_1", module=nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size))
    self.attention_pooling.add_module(name="att_act", module=nn.Tanh())
    self.attention_pooling.add_module(name="att_fc_out", module=nn.Linear(in_features=self.hidden_size, out_features=1, bias=False))
    self.attention_pooling.add_module(name="att_softmax", module=nn.Softmax(dim=self.hidden_dim))

    # Final calculation of attention
    self.att_fc_in_2 = nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)
    self.att_act = nn.Tanh().to(device)

    # Polarity classifier
    self.fc_out = nn.Linear(in_features=self.hidden_size, out_features=self.polarity_num)

  def forward(self, last_hidden_states_masked):
    """
    Aspect attention pooling
    att_alpha.shape is torch.Size([4, 512, 1])

    att_W2.shape is torch.Size([4, 512, 768])
    att_W2_T.shape is torch.Size([4, 768, 512])
    att_out.shape is torch.Size([4, 768, 1])
    att_r.shape is torch.Size([4, 768, 1])
    att_r_flat.shape is torch.Size([4, 768])
    """
    att_alpha = self.attention_pooling(last_hidden_states_masked)
    att_W2 = self.att_fc_in_2(last_hidden_states_masked)
    # print(f"att_W2.shape is {att_W2.shape}")
    att_W2_T = att_W2.view(-1, self.hidden_size, self.max_seq_len)
    # print(f"att_W2_T.shape is {att_W2_T.shape}")
    att_out = torch.matmul(att_W2_T, att_alpha)
    # print(f"att_out.shape is {att_out.shape}")
    att_r = self.att_act(att_out)
    # print(f"att_r.shape is {att_r.shape}")
    att_r_flat = att_r.view(-1, self.hidden_size)
    # print(f"att_r_flat.shape is {att_r_flat.shape}")
    # print()

    """
    Polarity classifier 
    logits.shape is torch.Size([4, 4])
    """ 
    logits = self.fc_out(att_r_flat)
    # print(f"logits.shape is {logits.shape}")
    
    return logits

class AspectSentimentAnalysis_main(nn.Module):
  def __init__(self, device, pretrain_bert_name, max_seq_len, aspect_num, polarity_num):
    super().__init__()

    # Bert feature extractor
    self.device = device
    self.bert = BertModel.from_pretrained(pretrain_bert_name)
    self.max_seq_len = max_seq_len
    self.aspect_num = aspect_num
    self.hidden_size = 768
    self.hidden_dim = -1
    self.batch_repeat = 1
    self.review_repeat = 1
    self.polarity_num = polarity_num
    self.word_dim = 2

    """
    20220605 Andy
    不可用單純的python list存放module！這樣會無法正常訓練。
    multi-branch情況下，必須使用ModuleList。
    可觀察print(model)的結果，確認module是否被納入。
    When should I use nn.ModuleList and when should I use nn.Sequential
    https://discuss.pytorch.org/t/when-should-i-use-nn-modulelist-and-when-should-i-use-nn-sequential/5463/4
    """
    # Aspect polarity classifier
    self.aspect_branch_list = nn.ModuleList()
    for aspect in range(self.aspect_num):
      self.aspect_branch_list.append(AspectSentimentAnalysis_branch(max_seq_len=max_seq_len, polarity_num=polarity_num).to(self.device))

  def forward(self, input_ids, attention_mask):
    """
    Forward batch through bert
    last_hidden_states.shape is torch.Size([4, 512, 768])
    """
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
    last_hidden_states = outputs.last_hidden_state
    pooler_output  = outputs.pooler_output 
    # print(f"last_hidden_states.shape is {last_hidden_states.shape}")
    # print(f"last_hidden_states is {last_hidden_states}")
    # print()

    """
    Apply attention_mask to last_hidden_states
    attention_mask.shape is torch.Size([4, 512])
    attention_mask_unsqueeze.shape is torch.Size([4, 512, 1])
    attention_mask_repeat.shape is torch.Size([4, 512, 768])
    last_hidden_states_masked.shape is torch.Size([4, 512, 768])
    """
    batch_len, review_len, word_len = last_hidden_states.shape
    attention_mask_unsqueeze = torch.unsqueeze(attention_mask, dim=self.word_dim)
    # print(f"attention_mask_unsqueeze.shape is {attention_mask_unsqueeze.shape}")
    attention_mask_repeat = attention_mask_unsqueeze.repeat(self.batch_repeat, self.review_repeat, self.hidden_size)
    # print(f"attention_mask_repeat.shape is {attention_mask_repeat.shape}")
    # print(f"attention_mask_repeat[0] is {attention_mask_repeat[0]}")
    last_hidden_states_masked = last_hidden_states * attention_mask_repeat
    # print(f"last_hidden_states_masked.shape is {last_hidden_states_masked.shape}")
    # print(f"last_hidden_states_masked[0] is {last_hidden_states_masked[0]}")
    # print()    

    """
    Aspect polarity classifier 
    """ 
    aspect_logits_list = []
    for aspect in range(self.aspect_num):
      # logits.shape is torch.Size([4, 4])
      logits = self.aspect_branch_list[aspect](last_hidden_states_masked)
      # print(f"logits.shape is {logits.shape}")
      aspect_logits_list.append(logits)
    
    return aspect_logits_list

In [ ]:
model_path_list = [
  "/content/drive/MyDrive/Colab Notebooks/model_aspect_polarity_balanced_multi_branch.pt.pt 的副本.pt 的副本.pt 的副本", # 0.939, 無augmentation
  "/content/drive/MyDrive/Colab Notebooks/model_aspect_polarity_balanced_multi_branch.pt.pt 的副本.pt 的副本.pt 的副本", # 0.939, augmentation
  # "model_hw3_1_2nd_20211211-1245.pt", # 0.934, lr decay
]
"""
result: 
0.939 + 0.939 + 0.934 = 0.9386
0.939 + 0.939 = 0.943
"""
model_list = []
for model_path in model_path_list:
  print(f"Loading model {model_path}")
  model_base = AspectSentimentAnalysis_main(device=device, pretrain_bert_name=global_config["BERT_model"], max_seq_len=global_config["max_seq_len"], aspect_num=18, polarity_num=len(POLARITY_TXT))
  model_base = model_base.to(device)
  model_base.load_state_dict(torch.load(model_path))
  model_list.append(model_base)
  print()

Loading model /content/drive/MyDrive/Colab Notebooks/model_aspect_polarity_balanced_multi_branch.pt.pt 的副本.pt 的副本.pt 的副本


Downloading:   0%|          | 0.00/659 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]


Loading model /content/drive/MyDrive/Colab Notebooks/model_aspect_polarity_balanced_multi_branch.pt.pt 的副本.pt 的副本.pt 的副本



In [ ]:
params_dict_sum = collections.OrderedDict()
params_keys = None
params_dict_averaged = collections.OrderedDict()

def AddModelPara2Dict(model, model_path):
  global params_dict_sum
  global params_keys

  # Get keys from model
  model_state_dict = model.state_dict()
  model_params_keys = list(model_state_dict.keys())
  if params_keys is None:
    params_keys = model_params_keys
  elif params_keys != model_params_keys:
    raise KeyError(
        "For checkpoint {}, expected list of params: {}, "
        "but found: {}".format(model_path, params_keys, model_params_keys)
    )

  """
  ['cnn_layers.0.weight', 'cnn_layers.0.bias', 'cnn_layers.1.weight', 'cnn_layers.1.bias', 'cnn_layers.1.running_mean', 'cnn_layers.1.running_var', 'cnn_layers.1.num_batches_tracked', 'cnn_layers.4.weight', 'cnn_layers.4.bias', 'cnn_layers.5.weight', 'cnn_layers.5.bias', 'cnn_layers.5.running_mean', 'cnn_layers.5.running_var', 'cnn_layers.5.num_batches_tracked', 'cnn_layers.8.weight', 'cnn_layers.8.bias', 'cnn_layers.9.weight', 'cnn_layers.9.bias', 'cnn_layers.9.running_mean', 'cnn_layers.9.running_var', 'cnn_layers.9.num_batches_tracked', 
  'fc_layers.0.weight', 'fc_layers.0.bias', 'fc_layers.2.weight', 'fc_layers.2.bias', 'fc_layers.4.weight', 'fc_layers.4.bias']
  """
  # Sum parameters of different models
  for k in params_keys:
    p = model_state_dict[k]
    if isinstance(p, torch.HalfTensor):
      p = p.float()
    if k not in params_dict_sum:
      params_dict_sum[k] = p.clone()
      # NOTE: clone() is needed in case of p is a shared parameter
    else:
      print(f"Append {k}")
      params_dict_sum[k] += p


# Ensemble can only be performed on the same models

# Add parameters of model to params_dict
ensemble_len = 0
for i, model in enumerate(model_list):
  print(f"Adding model {model_path_list[i]}")
  AddModelPara2Dict(model, model_path_list[i])
  ensemble_len += 1

# Average parameters
for k, v in params_dict_sum.items():
  params_dict_averaged[k] = v
  print(f"Divide {k}")
  if params_dict_averaged[k].is_floating_point():
    params_dict_averaged[k].div_(ensemble_len)
  else:
    params_dict_averaged[k] //= ensemble_len

# Set parameters back to a new model
model_ensemble = AspectSentimentAnalysis_main(device=device, pretrain_bert_name=global_config["BERT_model"], max_seq_len=global_config["max_seq_len"], aspect_num=18, polarity_num=len(POLARITY_TXT))
model_ensemble.to(device)
model_ensemble.load_state_dict(params_dict_averaged) 

# Save ensemble model
# for k, v in averaged_params:
#   model_ensemble[k] = v  
model_path_ensemble = "NEW_model.ckpt"
model_path = model_path_ensemble
print(f"Saving model {model_path_ensemble}")
torch.save(model_ensemble.state_dict(), model_path_ensemble)
model_base = model_ensemble

Adding model /content/drive/MyDrive/Colab Notebooks/model_aspect_polarity_balanced_multi_branch.pt.pt 的副本.pt 的副本.pt 的副本
Adding model /content/drive/MyDrive/Colab Notebooks/model_aspect_polarity_balanced_multi_branch.pt.pt 的副本.pt 的副本.pt 的副本
Append bert.embeddings.position_ids
Append bert.embeddings.word_embeddings.weight
Append bert.embeddings.position_embeddings.weight
Append bert.embeddings.token_type_embeddings.weight
Append bert.embeddings.LayerNorm.weight
Append bert.embeddings.LayerNorm.bias
Append bert.encoder.layer.0.attention.self.query.weight
Append bert.encoder.layer.0.attention.self.query.bias
Append bert.encoder.layer.0.attention.self.key.weight
Append bert.encoder.layer.0.attention.self.key.bias
Append bert.encoder.layer.0.attention.self.value.weight
Append bert.encoder.layer.0.attention.self.value.bias
Append bert.encoder.layer.0.attention.output.dense.weight
Append bert.encoder.layer.0.attention.output.dense.bias
Append bert.encoder.layer.0.attention.output.LayerNorm.wei

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:590.)


Divide aspect_branch_list.14.fc_out.weight
Divide aspect_branch_list.14.fc_out.bias
Divide aspect_branch_list.15.attention_pooling.att_fc_in_1.weight
Divide aspect_branch_list.15.attention_pooling.att_fc_in_1.bias
Divide aspect_branch_list.15.attention_pooling.att_fc_out.weight
Divide aspect_branch_list.15.att_fc_in_2.weight
Divide aspect_branch_list.15.att_fc_in_2.bias
Divide aspect_branch_list.15.fc_out.weight
Divide aspect_branch_list.15.fc_out.bias
Divide aspect_branch_list.16.attention_pooling.att_fc_in_1.weight
Divide aspect_branch_list.16.attention_pooling.att_fc_in_1.bias
Divide aspect_branch_list.16.attention_pooling.att_fc_out.weight
Divide aspect_branch_list.16.att_fc_in_2.weight
Divide aspect_branch_list.16.att_fc_in_2.bias
Divide aspect_branch_list.16.fc_out.weight
Divide aspect_branch_list.16.fc_out.bias
Divide aspect_branch_list.17.attention_pooling.att_fc_in_1.weight
Divide aspect_branch_list.17.attention_pooling.att_fc_in_1.bias
Divide aspect_branch_list.17.attention_p